### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import GRU

from encrypt import *
from tools import *
import string, numpy

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.printable.split('!')[0]
encrypt = transposition_cipher()
samples_per_key = 200
number_of_keys = 1000
text_length = 12

set_characters(letters)
model_path = f"models/keyed_{encrypt.name}_{len(letters)}x{text_length}_best_model.h5"

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_per_key)
set_characters('12345678')
keys = generate_text(8, number_of_keys, unique=True)
ciphers = [list(map(encrypt(key), text)) for key in keys]

In [4]:
train_keys = to_vec(keys, False)
train_ciphers = numpy.array([to_vec(cipher, False) for cipher in ciphers])

### Building and Training the Model

In [5]:
model = Sequential()
model.add(Bidirectional(GRU(128, activation='relu'), input_shape=train_ciphers.shape[1:]))
model.add(Dense(8, activation='linear'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 256)               108288    
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 2056      
Total params: 110,344
Trainable params: 110,344
Non-trainable params: 0
_________________________________________________________________


In [6]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=4, verbose=8, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_ciphers, train_keys, validation_split=.1, batch_size=1, epochs=50, callbacks=callbacks)

Train on 900 samples, validate on 100 samples
Epoch 1/50
900/900 [==============================] - 70s 78ms/step - loss: 5.3184 - acc: 0.4556 - val_loss: 2.0041 - val_acc: 0.5500

Epoch 00001: val_acc improved from -inf to 0.55000, saving model to models/keyed_transposition_cipher_62x12_best_model.h5
Epoch 2/50
900/900 [==============================] - 70s 78ms/step - loss: 1.5509 - acc: 0.6433 - val_loss: 1.2382 - val_acc: 0.5700

Epoch 00002: val_acc improved from 0.55000 to 0.57000, saving model to models/keyed_transposition_cipher_62x12_best_model.h5
Epoch 3/50
900/900 [==============================] - 70s 77ms/step - loss: 1.0430 - acc: 0.6556 - val_loss: 0.9152 - val_acc: 0.5700

Epoch 00003: val_acc did not improve from 0.57000
Epoch 4/50
900/900 [==============================] - 71s 79ms/step - loss: 0.8534 - acc: 0.6544 - val_loss: 0.8819 - val_acc: 0.6500

Epoch 00004: val_acc improved from 0.57000 to 0.65000, saving model to models/keyed_transposition_cipher_62x12_best_m

### Predicting and Evaluating the Model

In [9]:
model = load_model(model_path)

In [10]:
percent = 0

for key, cipher in zip(keys, train_ciphers):
    prediction = to_txt(model.predict(cipher.reshape((1, samples_per_key, text_length))))
    percent += match_percentage(prediction, key)

print(f'Average accurecy: {percent / len(keys):.3}%')

Average accurecy: 80.8%
